In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/accidents-us-f/Accidents_US_F
/kaggle/input/us-accidents/US_Accidents_March23.csv


In [2]:
data = pd.read_csv("/kaggle/input/us-accidents/US_Accidents_March23.csv")

# EDA

In [3]:
data.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
data.nunique()

ID                       7728394
Source                         3
Severity                       4
Start_Time               6131796
End_Time                 6705355
Start_Lat                2428358
Start_Lng                2482533
End_Lat                  1568172
End_Lng                  1605789
Distance(mi)               22382
Description              3761578
Street                    336306
City                       13678
County                      1871
State                         49
Zipcode                   825094
Country                        1
Timezone                       4
Airport_Code                2045
Weather_Timestamp         941331
Temperature(F)               860
Wind_Chill(F)               1001
Humidity(%)                  100
Pressure(in)                1144
Visibility(mi)                92
Wind_Direction                24
Wind_Speed(mph)              184
Precipitation(in)            299
Weather_Condition            144
Amenity                        2
Bump      

In [5]:
data = data.drop_duplicates()

In [6]:
data.shape

(7728394, 46)

In [8]:
data.isnull().sum()


ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
City                         253
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump      

# Preprocessing

In [9]:
to_drop = ["ID",
 'Source',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Airport_Code',
 'Wind_Direction',
 'Weather_Timestamp',
 'Wind_Chill(F)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'No_Exit',
 'Turning_Loop',
 'Traffic_Calming',
 'Roundabout',
 'Bump',
 'Timezone',
 'Amenity',
 "Civil_Twilight", 
 "Nautical_Twilight",
 "Astronomical_Twilight"]

In [10]:
df = data.drop(to_drop,axis=1)

In [15]:
df = df.dropna(subset="Start_Time")

In [13]:
import datetime

In [17]:
df['Start_Time'] = pd.to_datetime(df.Start_Time,format="ISO8601")

In [18]:
df['Start_Time_year'] = df['Start_Time'].dt.year
df['Start_Time_month'] = df['Start_Time'].dt.month
df['Start_Time_day'] = df['Start_Time'].dt.day
df['Start_Time_hour'] = df['Start_Time'].dt.hour
df['Start_Time_minute'] = df['Start_Time'].dt.minute
df['Start_Time_second'] = df['Start_Time'].dt.second

In [19]:
df.drop("Start_Time",axis=1,inplace=True)

In [23]:
df.shape

(7728394, 22)

In [24]:
df = df.dropna()

In [25]:
temp = df.select_dtypes(include="int32")

In [26]:
df[['Start_Time_year', 'Start_Time_month', 'Start_Time_day',
       'Start_Time_hour', 'Start_Time_minute', 'Start_Time_second']] = df[['Start_Time_year', 'Start_Time_month', 'Start_Time_day','Start_Time_hour', 'Start_Time_minute', 'Start_Time_second']].astype(int)

In [27]:
# df.drop("Unnamed: 0",axis=1,inplace=True)

In [28]:
# df.to_csv("Accident.csv")

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt


In [30]:
# numaric = df.select_dtypes(include=["int64","float64"])
# for num in numaric:
#     sns.histplot(df[num], kde=True)
#     plt.show()
#     sns.boxplot(df[num])
#     plt.show()

# Label Encoding to convert ordinal values to numerical data 

In [33]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

lb = LabelEncoder()

df["Sunrise_Sunset"] = lb.fit_transform(df["Sunrise_Sunset"])

In [34]:
import joblib
# joblib.dump(lb, "label.joblib")


['label.joblib']

In [39]:
df = pd.get_dummies(df,prefix=None)

In [43]:
temp = df.select_dtypes("bool").columns

In [44]:
df[temp] = df[temp].astype(int)

# UnderSampled data using RandomSampler

In [126]:
df = pd.read_csv("/kaggle/input/accidents-us-f/Accidents_US_F")

In [127]:
df.shape

(248376, 117)

In [128]:
df = df.drop("Unnamed: 0",axis=1)

# Train Test Split

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
X = df.drop("Severity",axis=1)
Y = df['Severity']

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify=Y, test_size=0.25, random_state = 123)

# Scaling numerical columns

In [134]:
from sklearn.preprocessing import StandardScaler
cols = ["Temperature(F)","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Speed(mph)","Precipitation(in)"]


In [138]:
scaler = StandardScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])

['scaler.joblib']

In [143]:
# joblib.dump(scaler, "scaler.bin")

['scaler.bin']

In [139]:
X_test[cols]=scaler.transform(X_test[cols])

# Model Building

In [57]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report

In [58]:
EM = pd.DataFrame(columns=['Name','accuracy'])
def appendEM(Name,y_acc,y_pred):
  EM.loc[len(EM)] = [Name,accuracy_score(y_acc,y_pred)]

## Decision Tree Classifier

In [59]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier()
dtree_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [60]:
y_test_pred = dtree_model.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.81      0.80      0.80     15523
           2       0.48      0.47      0.47     15524
           3       0.51      0.50      0.50     15524
           4       0.47      0.49      0.48     15523

    accuracy                           0.57     62094
   macro avg       0.57      0.57      0.57     62094
weighted avg       0.57      0.57      0.57     62094



In [61]:
appendEM('DecisionTreeClassifier',y_test,y_test_pred)
EM

,Name,accuracy
0,DecisionTreeClassifier,0.565079


## Logistic Regression

In [62]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [63]:
y_test_pred = lr.predict(X_test)
appendEM('LogisticRegression',y_test,y_test_pred)
EM

,Name,accuracy
0,DecisionTreeClassifier,0.565079
1,LogisticRegression,0.369311


## Random Forest Classifier

In [64]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [65]:
y_test_pred = rf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.78      0.93      0.85     15523
           2       0.60      0.51      0.55     15524
           3       0.57      0.72      0.64     15524
           4       0.64      0.46      0.54     15523

    accuracy                           0.65     62094
   macro avg       0.65      0.65      0.64     62094
weighted avg       0.65      0.65      0.64     62094



In [66]:
appendEM('RandomForestClassifier',y_test,y_test_pred)
EM

,Name,accuracy
0,DecisionTreeClassifier,0.565079
1,LogisticRegression,0.369311
2,RandomForestClassifier,0.652430


# XGBoost

In [67]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train-1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [68]:
y_test_pred = xgb_clf.predict(X_test)
print(classification_report(y_test-1, y_test_pred))


              precision    recall  f1-score   support

           0       0.75      0.95      0.84     15523
           1       0.62      0.49      0.55     15524
           2       0.57      0.74      0.64     15524
           3       0.60      0.39      0.48     15523

    accuracy                           0.64     62094
   macro avg       0.64      0.64      0.63     62094
weighted avg       0.64      0.64      0.63     62094



In [69]:
appendEM('xgboostClassifier',y_test-1,y_test_pred)
EM

,Name,accuracy
0,DecisionTreeClassifier,0.565079
1,LogisticRegression,0.369311
2,RandomForestClassifier,0.652430
3,xgboostClassifier,0.642300


# Increasing the recall of class '4' Severity (High Severity) using two models
### Making a binary classification to classify if the severity is 4 or not. If the prediction is not 4, it will be followed by predicted class 1,2,3
### This will help to increase the recall of class 4 as it the most severe case

In [70]:
X_t = X.copy()
y_t = Y.copy()

In [71]:
y_t = y_t.replace([1,2,3],0)
y_t = y_t.replace(4,1)

In [72]:
y_t.value_counts()

Severity
0    186282
1     62094
Name: count, dtype: int64

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y_t,stratify=y_t, test_size=0.25, random_state = 123)

In [74]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [75]:
proba = rf.predict_proba(X_test)
y_pred_new_threshold = (proba[:, 1] >= 0.19).astype(int) 


In [76]:
print(classification_report(y_test, y_pred_new_threshold))

              precision    recall  f1-score   support

           0       0.93      0.50      0.65     46571
           1       0.37      0.88      0.52     15523

    accuracy                           0.60     62094
   macro avg       0.65      0.69      0.59     62094
weighted avg       0.79      0.60      0.62     62094



In [78]:
df1 = df[df["Severity"]!=4]

In [79]:
X = df1.drop("Severity",axis=1)
Y = df1['Severity']

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,stratify=Y, test_size=0.25, random_state = 123)

In [81]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [82]:
y_test_pred = rf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           1       0.83      0.93      0.88     15523
           2       0.80      0.66      0.72     15524
           3       0.72      0.76      0.74     15524

    accuracy                           0.79     46571
   macro avg       0.79      0.79      0.78     46571
weighted avg       0.79      0.79      0.78     46571

